## Joins with `sqlalchemy`

To join to tables in `sqlalchemy`

1. Use `join` to create a `Join` object
2. Build a `select` statement from the `join` object

## Example - Reading in the Company `db`

In [1]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base

engine = create_engine("sqlite:///databases/company_2_7_2.db")

Base = automap_base()
Base.prepare(engine, reflect=True)
Dept = Base.classes.department
Empl = Base.classes.employee

In [3]:
import pandas as pd
from sqlalchemy import select as selectq
d = selectq([Dept])
pd.read_sql_query(d, con=engine)

,DeptID,DeptName
0,31,Sales
1,33,Engineering
2,34,Clerical
3,35,Marketing


In [6]:
e = selectq([Empl])
pd.read_sql_query(e, con=engine)

,DeptID,LastName,EmpID
0,31.0,Rafferty,1
1,33.0,Jones,2
2,33.0,Heisenberg,3
3,34.0,Robinson,4
4,34.0,Smith,5
5,NaN,Williams,6


## Using `sqlalchemy.join` to create a `Join`

**Syntax:** `join(left_table, right_table, onclause=left_table.column == right_table.column)`

* Defaults to an inner join
* Set `isouter=True` to get a `LEFT OUTER JOIN`
* Set `full=True` to get a `FULL OUTER JOIN`

In [7]:
from sqlalchemy import join
j = join(Empl, Dept, onclause=Empl.DeptID == Dept.DeptID)
print(j)

employee JOIN department ON employee."DeptID" = department."DeptID"


## Inspecting the joined column names

Note that the columns are renamed `tableName_columnName`.

In [8]:
j.c.keys()

['employee_DeptID',
 'employee_LastName',
 'employee_EmpID',
 'department_DeptID',
 'department_DeptName']

## Creating a `Select` expression for a `Join`

In [9]:
from sqlalchemy import func, select as selectq

stmt = selectq([j])
print(stmt)

SELECT employee."DeptID", employee."LastName", employee."EmpID", department."DeptID", department."DeptName" 
FROM employee JOIN department ON employee."DeptID" = department."DeptID"


In [10]:
pd.read_sql_query(stmt, con=engine)

,DeptID,LastName,EmpID,DeptID,DeptName
0,31.0,Rafferty,1,31,Sales
1,33.0,Jones,2,33,Engineering
2,33.0,Heisenberg,3,33,Engineering
3,34.0,Robinson,4,34,Clerical
4,34.0,Smith,5,34,Clerical


## Left Join

In [11]:
left_join = join(Empl, Dept, onclause=Empl.DeptID==Dept.DeptID, isouter=True)
left_join_stmt = selectq([left_join])
pd.read_sql_query(left_join_stmt, con=engine)

,DeptID,LastName,EmpID,DeptID,DeptName
0,31.0,Rafferty,1,31.0,Sales
1,33.0,Jones,2,33.0,Engineering
2,33.0,Heisenberg,3,33.0,Engineering
3,34.0,Robinson,4,34.0,Clerical
4,34.0,Smith,5,34.0,Clerical
5,NaN,Williams,6,NaN,None


## Right Join

To get a `RIGHT OUTER JOIN`, just switch the order and use a `LEFT OUTER JOIN`

In [12]:
right_join = join(Dept, Empl, onclause=Empl.DeptID==Dept.DeptID, isouter=True)
right_join_stmt = selectq([right_join])
pd.read_sql_query(right_join_stmt, con=engine)

,DeptID,DeptName,DeptID,LastName,EmpID
0,31,Sales,31.0,Rafferty,1.0
1,33,Engineering,33.0,Heisenberg,3.0
2,33,Engineering,33.0,Jones,2.0
3,34,Clerical,34.0,Robinson,4.0
4,34,Clerical,34.0,Smith,5.0
5,35,Marketing,NaN,None,NaN


## Full Outer Join

**Note:** `sqllite` does not support this type of join `:/`

In [13]:
full_join = join(Empl, Dept, onclause=Empl.DeptID==Dept.DeptID, full=True)
full_join_stmt = selectq([full_join])
pd.read_sql_query(full_join_stmt, con=engine)

OperationalError: (sqlite3.OperationalError) RIGHT and FULL OUTER JOINs are not currently supported [SQL: 'SELECT employee."DeptID", employee."LastName", employee."EmpID", department."DeptID", department."DeptName" \nFROM employee FULL OUTER JOIN department ON employee."DeptID" = department."DeptID"'] (Background on this error at: http://sqlalche.me/e/e3q8)

## <font color="red"> Exercise 3 </font>

Determine all the players that have hit more than 100 home runs in a season.  The final table should include the players proper name, as well as the team name.  

**Hint:** You will need join the files listed below.  To get credit for this exercise, you will need to create a database containing these three tables and use the `sqlalchemy` join methods presented above.

In [129]:
f1, f2, f3 = ("./data/baseball/core/Batting.csv", 
              "./data/baseball/core/People.csv",
              "./data/baseball/core/Teams.csv")

'./data/baseball/core/Teams.csv'

In [23]:
# Your code here

## Up Next

Stuff